In [ ]:
import Config as config
import tweepy
import preprocessor as p
import pykafka
from googletrans import Translator
p.set_options(p.OPT.URL, p.OPT.RESERVED, p.OPT.MENTION, p.OPT.EMOJI, p.OPT.HASHTAG,p.OPT.SMILEY)
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import json

In [ ]:
class CollectionDataTwitter(tweepy.StreamListener):
    
    def __init__(self,api,topic,limit=100):
        self.topic = topic
        self.limit = limit
        self.counter = 0
        self.client = pykafka.KafkaClient(config.bootstrap_servers)
        self.producer = self.client.topics[bytes(config.kafka_topic_name, config.data_encoding)].get_producer()
        super().__init__(api)
    
    def on_data(self,data):
        
        data = json.loads(data)
        tweet_text = p.clean(data['text'])
        
        if self.topic.lower() not in tweet_text.lower():
            return

      
        blob = Translator().translate(tweet_text)
        
        
        if SentimentIntensityAnalyzer().polarity_scores(blob.text)['compound'] > 0 :
            self.sentiment = "\t1\n"
        
        else: 
            self.sentiment = "\t0\n"
        
        #with open("data_file.txt",mode="+a") as data:
            #data.write(f"{tweet_text}{self.sentiment}")
        print(f"{tweet_text}{self.sentiment}")
        self.producer.produce(bytes(f"{tweet_text}{self.sentiment}", config.data_encoding))
        self.counter+=1
        return self.counter != self.limit 
    
    
    def on_error(self, status):        
        print(status)            
        return False
        
    
        
        

In [ ]:
if __name__=="__main__":
    
    auth = tweepy.OAuthHandler(config.consumer_key, config.consumer_secret)
    auth.set_access_token(config.access_token, config.access_secret)
    api = tweepy.API(auth)
    stream = tweepy.Stream(auth=api.auth,listener=CollectionDataTwitter(api,config.twitter_stream_filter))
    stream.filter(track=[config.twitter_stream_filter],languages=['fr'],is_async=False)
    
             